# Comparing channels

## Nucleotides
- R: sequential
- G: complementary
- B: not match at all

## Proteins
- sequential by BLOSUM62

In [6]:
import ppscore
import pandas
import os
import numpy
from scipy.stats import spearmanr
from IPython.core.display import display, HTML
from ete3 import Tree
from ete3.parser.newick import NewickError

/tmp/ipykernel_1482554/1860748529.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [21]:
tree_path = "../../data/trees"
channels = ("N/full", "N/gray_r", "N/gray_g", "N/gray_b", "P/full")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin", "indelible"]

In [8]:
def do_euclidean(control, compare):
    return round(numpy.sqrt(numpy.sum((control.values - compare.values)**2)), 4)

def do_spearman(control, compare):
    return round(spearmanr(control.values.flatten(), compare.values.flatten())[0], 4)

def do_ppscore(control, compare):
    return round(ppscore.score(
                    pandas.DataFrame({"x": control.values.flatten(), "y": compare.values.flatten()}), "x", "y")["ppscore"], 4)

def do_rf(control, compare):
    return round(control.compare(compare, unrooted=True)["norm_rf"], 4)

def do_branch_score(control, compare):
    return 1.0 - round(control.compare(compare, unrooted=True)["source_edges_in_ref"], 4)


def read_and_compare(dataset, channel, score_func, control_name):
    result_dict = dict(
        algorithm=[],
        score=[],
        channel=[],
        dataset=[]
    )
    path = f"{tree_path}/{channel}/{dataset}"
    ends = control_name.split(".")[-1]
    if ends == "nw":
        control = Tree(f"{path}/{control_name}", format=1)
    else:
        control = pandas.read_csv(f"{path}/{control_name}", index_col=0)
    for file in os.listdir(path):
        if file.endswith(ends):
            basename = ".".join(file.split(".")[0:-1])
            try:
                if ends == "nw":
                    compare = Tree(f"{path}/{file}", format=1)
                else:
                    compare = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict["algorithm"] += [basename]
                result_dict["score"] += [score_func(control, compare)]
                result_dict["channel"] += [channel]
                result_dict["dataset"] += [dataset]
            except (FileNotFoundError, NewickError) as e:
                print(f"Error with file {path}/{file}")
    return result_dict

def pprintdf(score_func, control_name="Control with Clustal Omega.csv"):
    result_dict = dict(
        algorithm=[],
        score=[],
        channel=[],
        dataset=[]
    )
    for channel in channels:
        for dataset in datasets:
            d = read_and_compare(dataset, channel, score_func, control_name)
            result_dict["algorithm"] += d["algorithm"]
            result_dict["score"] += d["score"]        
            result_dict["channel"] += d["channel"]        
            result_dict["dataset"] += d["dataset"]
    df = pandas.DataFrame(result_dict)
    table = df.pivot(index=["algorithm", "channel"], columns="dataset", values="score")
    return table.head(50)

## Euclidean Distance from distance matrices

In [12]:
pprintdf(do_euclidean)

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       0.0000   
                                                   N/gray_b     0.0000   
                                                   N/gray_g     0.0000   
                                                   N/gray_r     0.0000   
                                                   P/full       0.0000   
Deep Search with Annoy                             N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
Global with Needleman-Wunsch                       N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       5.4879   
Greedy Sliced Structural Similarity Index Measure  N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       1.3967   
Local with Smith–Waterman                          N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       4.3137   
Resized MultiScale Structural Similarity Index ... N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full      12.2543   
Resized Structural Similarity Index Measure        N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       1.3972   
Universal Quality Index                            N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full      22.4514   
Unrestricted Sliced Structural Similarity Index... N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       1.3850   
Windowed MultiScale Structural Similarity Index... N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full      12.2543   

dataset                                                      orthologs_androglobin  \
algorithm                                          channel                           
Control with Clustal Omega                         N/

## Linear Correlation from distance matrices

In [13]:
pprintdf(do_spearman)

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       1.0000   
                                                   N/gray_b     1.0000   
                                                   N/gray_g     1.0000   
                                                   N/gray_r     1.0000   
                                                   P/full       1.0000   
Deep Search with Annoy                             N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
Global with Needleman-Wunsch                       N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.3039   
Greedy Sliced Structural Similarity Index Measure  N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.2122   
Local with Smith–Waterman                          N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.2445   
Resized MultiScale Structural Similarity Index ... N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1534   
Resized Structural Similarity Index Measure        N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.2105   
Universal Quality Index                            N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.2680   
Unrestricted Sliced Structural Similarity Index... N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.2166   
Windowed MultiScale Structural Similarity Index... N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1534   

dataset                                                      orthologs_androglobin  \
algorithm                                          channel                           
Control with Clustal Omega                         N/

## Predictive Power Score from distance matrices

In [14]:
pprintdf(do_ppscore)

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       0.9998   
                                                   N/gray_b     0.9998   
                                                   N/gray_g     0.9998   
                                                   N/gray_r     0.9998   
                                                   P/full       0.9992   
Deep Search with Annoy                             N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
Global with Needleman-Wunsch                       N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.8767   
Greedy Sliced Structural Similarity Index Measure  N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.9131   
Local with Smith–Waterman                          N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.9643   
Resized MultiScale Structural Similarity Index ... N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.7838   
Resized Structural Similarity Index Measure        N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.9116   
Universal Quality Index                            N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.7538   
Unrestricted Sliced Structural Similarity Index... N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.9158   
Windowed MultiScale Structural Similarity Index... N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.7838   

dataset                                                      orthologs_androglobin  \
algorithm                                          channel                           
Control with Clustal Omega                         N/

## Robinson-foulds distance from generated Trees

In [15]:
pprintdf(do_rf, control_name="Control with Clustal Omega.nw")

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       0.0000   
                                                   N/gray_b     0.0000   
                                                   N/gray_g     0.0000   
                                                   N/gray_r     0.0000   
                                                   P/full       0.0000   
Deep Search with Annoy                             N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
Global with Needleman-Wunsch                       N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1892   
Greedy Sliced Structural Similarity Index Measure  N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1892   
Local with Smith–Waterman                          N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1892   
Resized MultiScale Structural Similarity Index ... N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1892   
Resized Structural Similarity Index Measure        N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1892   
Universal Quality Index                            N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1892   
Unrestricted Sliced Structural Similarity Index... N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1892   
Windowed MultiScale Structural Similarity Index... N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1892   

dataset                                                      orthologs_androglobin  \
algorithm                                          channel                           
Control with Clustal Omega                         N/

In [19]:
pprintdf(do_rf, control_name="Global with Needleman-Wunsch.nw")

dataset                                                      orthologs_androglobin  \
algorithm                                          channel                           
Control with Clustal Omega                         N/full                   0.8333   
                                                   N/gray_b                 0.8333   
                                                   N/gray_g                 0.8333   
                                                   N/gray_r                 0.8333   
                                                   P/full                   0.9167   
Deep Search with Annoy                             N/full                      NaN   
                                                   N/gray_b                    NaN   
                                                   N/gray_g                    NaN   
                                                   N/gray_r                    NaN   
Global with Needleman-Wunsch                       N/full                   0.0000   
                                                   N/gray_b                 0.0000   
                                                   N/gray_g                 0.0000   
                                                   N/gray_r                 0.0000   
                                                   P/full                   0.0000   
Greedy Sliced Structural Similarity Index Measure  N/full                      NaN   
                                                   N/gray_b                    NaN   
                                                   N/gray_g                    NaN   
                                                   N/gray_r                    NaN   
                                                   P/full                   0.7500   
Local with Smith–Waterman                          N/full                   0.4167   
                                                   N/gray_b                 0.4167   
                                                   N/gray_g                 0.4167   
                                                   N/gray_r                 0.4167   
                                                   P/full                   0.3333   
Resized MultiScale Structural Similarity Index ... N/full                   0.5000   
                                                   N/gray_b                 0.5000   
                                                   N/gray_g                 0.5000   
                                                   N/gray_r                 0.5000   
                                                   P/full                   0.5000   
Resized Structural Similarity Index Measure        N/full                   0.5833   
                                                   N/gray_b                 0.5833   
                                                   N/gray_g                 0.5833   
                                                   N/gray_r                 0.5833   
                                                   P/full                   0.5833   
Universal Quality Index                            N/full                      NaN   
                                                   N/gray_b                    NaN   
                                                   N/gray_g                    NaN   
                                                   N/gray_r                    NaN   
                                                   P/full                   0.4167   
Unrestricted Sliced Structural Similarity Index... N/full                      NaN   
                                                   N/gray_b                    NaN   
                                                   N/gray_g                    NaN   
                                                   N/gray_r                    NaN   
                                                   P/full                   0.8333   
Windowed MultiScale Structural Similarity In

In [20]:
pprintdf(do_rf, control_name="Local with Smith–Waterman.nw")

dataset                                                      orthologs_androglobin  \
algorithm                                          channel                           
Control with Clustal Omega                         N/full                   0.5000   
                                                   N/gray_b                 0.5000   
                                                   N/gray_g                 0.5000   
                                                   N/gray_r                 0.5000   
                                                   P/full                   0.6667   
Deep Search with Annoy                             N/full                      NaN   
                                                   N/gray_b                    NaN   
                                                   N/gray_g                    NaN   
                                                   N/gray_r                    NaN   
Global with Needleman-Wunsch                       N/full                   0.4167   
                                                   N/gray_b                 0.4167   
                                                   N/gray_g                 0.4167   
                                                   N/gray_r                 0.4167   
                                                   P/full                   0.3333   
Greedy Sliced Structural Similarity Index Measure  N/full                      NaN   
                                                   N/gray_b                    NaN   
                                                   N/gray_g                    NaN   
                                                   N/gray_r                    NaN   
                                                   P/full                   0.6667   
Local with Smith–Waterman                          N/full                   0.0000   
                                                   N/gray_b                 0.0000   
                                                   N/gray_g                 0.0000   
                                                   N/gray_r                 0.0000   
                                                   P/full                   0.0000   
Resized MultiScale Structural Similarity Index ... N/full                   0.5833   
                                                   N/gray_b                 0.5833   
                                                   N/gray_g                 0.5833   
                                                   N/gray_r                 0.5833   
                                                   P/full                   0.5833   
Resized Structural Similarity Index Measure        N/full                   0.6667   
                                                   N/gray_b                 0.6667   
                                                   N/gray_g                 0.6667   
                                                   N/gray_r                 0.6667   
                                                   P/full                   0.6667   
Universal Quality Index                            N/full                      NaN   
                                                   N/gray_b                    NaN   
                                                   N/gray_g                    NaN   
                                                   N/gray_r                    NaN   
                                                   P/full                   0.5833   
Unrestricted Sliced Structural Similarity Index... N/full                      NaN   
                                                   N/gray_b                    NaN   
                                                   N/gray_g                    NaN   
                                                   N/gray_r                    NaN   
                                                   P/full                   0.8333   
Windowed MultiScale Structural Similarity In

## Compatibility branch score from generated Trees

In [22]:
pprintdf(do_branch_score, control_name="Control with Clustal Omega.nw")

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       0.0000   
                                                   N/gray_b     0.0000   
                                                   N/gray_g     0.0000   
                                                   N/gray_r     0.0000   
                                                   P/full       0.0000   
Deep Search with Annoy                             N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
Global with Needleman-Wunsch                       N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.0921   
Greedy Sliced Structural Similarity Index Measure  N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.0921   
Local with Smith–Waterman                          N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.0921   
Resized MultiScale Structural Similarity Index ... N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.0921   
Resized Structural Similarity Index Measure        N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.0921   
Universal Quality Index                            N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.0921   
Unrestricted Sliced Structural Similarity Index... N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.0921   
Windowed MultiScale Structural Similarity Index... N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.0921   

dataset                                                      orthologs_androglobin  \
algorithm                                          channel                           
Control with Clustal Omega                         N/